Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # slick way to just drop a dimension. presumably this knows to loose last dim
    
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    # np.arrange returns evenly spaced values within a given interval: np.arange(3) -> array([0, 1, 2])
    # so, train_labels are [200000,] array w/ values 0-9 (or 1-10). this is taking all the lables:
    # (labels[:,None]), finding where they equal the num_labels (0-9 array), an, i guess flipping a flag.
    # so, it goes from "9" to array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.], dtype=float32)
    
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
train_labels[1]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.], dtype=float32)

We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
# above loads in tf graph function. below feeds it input data, defines variables and sets parms
with graph.as_default():
    # Input data.
    # Load the training, validation and test data into constants that are attached to the graph
    # parently in tf, the input data is considered a contstnt (relative to processing vars)
    # looks like it's auto-subsetting the data based on train_subset = 1000
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    # truncated_normal must be rnorm()
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases to get pred "y" (or logits). 
    # We compute the softmax (S(y)) and cross-entropy (after comparing to i-hot labels). It's one 
    # operation in TensorFlow, because it's very common, and it can be optimized). We take the 
    # average of this cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    # so, this is just a matrix multiplication of the X values and the weights followed by matrix add
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    # then input logits, perform softmax, get preds, calculate loss, then average losses
    
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    # so, GDO is a function finds the gradient for each iteration. optimizer appears to be a tf setting.
    
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [6]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])
# simply counting instances where preds = lables and dividing by total rows

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the biases. 
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy arrays.
        
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
            
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 14.567147
Training accuracy: 12.3%
Validation accuracy: 15.1%
Loss at step 100: 2.412139
Training accuracy: 71.4%
Validation accuracy: 70.2%
Loss at step 200: 1.948093
Training accuracy: 74.2%
Validation accuracy: 72.5%
Loss at step 300: 1.689893
Training accuracy: 75.7%
Validation accuracy: 73.3%
Loss at step 400: 1.515301
Training accuracy: 76.4%
Validation accuracy: 73.9%
Loss at step 500: 1.385800
Training accuracy: 77.2%
Validation accuracy: 74.2%
Loss at step 600: 1.284233
Training accuracy: 77.8%
Validation accuracy: 74.4%
Loss at step 700: 1.201442
Training accuracy: 78.2%
Validation accuracy: 74.7%
Loss at step 800: 1.132199
Training accuracy: 78.7%
Validation accuracy: 75.0%
Test accuracy: 82.9%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `sesion.run()`.

In [7]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch. FIRST MENTION OF MINIBATCHES
    # to their iterating thru the dataset using minibatches of 128 records.
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    # note here and before, to get preds, you do the matmul using the validation set and the final
    # training weights and biases
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 13.780415
Minibatch accuracy: 18.0%
Validation accuracy: 17.1%
Minibatch loss at step 500: 1.428131
Minibatch accuracy: 75.8%
Validation accuracy: 74.7%
Minibatch loss at step 1000: 1.056165
Minibatch accuracy: 78.1%
Validation accuracy: 76.1%
Minibatch loss at step 1500: 1.529214
Minibatch accuracy: 74.2%
Validation accuracy: 77.0%
Minibatch loss at step 2000: 1.176421
Minibatch accuracy: 78.9%
Validation accuracy: 77.4%
Minibatch loss at step 2500: 0.969873
Minibatch accuracy: 78.1%
Validation accuracy: 78.1%
Minibatch loss at step 3000: 0.994329
Minibatch accuracy: 76.6%
Validation accuracy: 78.5%
Test accuracy: 86.0%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units (nn.relu()) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

NOTES:
So, we moved from logistic regression, to:
1. multinomial logistic regression using simple gradient descent: 83%
2. multinomial logistic regression using stochastic gradient descent: 86%

Below we move to:
1. multinomial neural net w/ 1 hidden layer using simple gradient descent: 86.3%
2. multinomial neural net w/ 1 hidden layer using stochastic gradient descent: 89%

In [9]:
# Subset the training data for faster turnaround.
train_subset = 10000
hidden1_units = 1024

graph = tf.Graph()
# above loads in tf graph function. below feeds it input data, defines variables and sets parms
with graph.as_default():
    # Input data.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    ## hidden1
    h1_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden1_units]))
    h1_biases = tf.Variable(tf.zeros([hidden1_units]))

    ## out1 - softmax linear
    out1_weights = tf.Variable(tf.truncated_normal([hidden1_units, num_labels]))
    out1_biases = tf.Variable(tf.zeros([num_labels]))
    
    ## iterative fit
    def forward_prop(input_obs):
        hidden1 = tf.nn.relu(tf.matmul(input_obs, h1_weights) + h1_biases)
        return tf.matmul(hidden1, out1_weights) + out1_biases
    
    # Training computation.
    logits = forward_prop(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset))
    test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset))

In [10]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # Initialize graph weights and biases 
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):        
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
            
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 425.172943
Training accuracy: 7.6%
Validation accuracy: 45.4%
Loss at step 100: 3.197669
Training accuracy: 91.6%
Validation accuracy: 78.2%
Loss at step 200: 0.207553
Training accuracy: 99.3%
Validation accuracy: 78.3%
Loss at step 300: 0.176121
Training accuracy: 98.3%
Validation accuracy: 76.6%
Loss at step 400: 0.037115
Training accuracy: 99.8%
Validation accuracy: 77.5%
Loss at step 500: 0.087704
Training accuracy: 99.8%
Validation accuracy: 78.7%
Loss at step 600: 0.064128
Training accuracy: 99.8%
Validation accuracy: 78.5%
Loss at step 700: 0.043733
Training accuracy: 99.8%
Validation accuracy: 77.5%
Loss at step 800: 0.085208
Training accuracy: 99.9%
Validation accuracy: 78.7%
Test accuracy: 86.3%


In [11]:
batch_size = 128
hidden1_units = 1024

graph = tf.Graph()
with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    ## hidden1
    h1_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden1_units]))
    h1_biases = tf.Variable(tf.zeros([hidden1_units]))

    ## out1 - softmax linear
    out1_weights = tf.Variable(tf.truncated_normal([hidden1_units, num_labels]))
    out1_biases = tf.Variable(tf.zeros([num_labels]))
    
    ## iterative fit
    def forward_prop(input_obs):
        hidden1 = tf.nn.relu(tf.matmul(input_obs, h1_weights) + h1_biases)
        return tf.matmul(hidden1, out1_weights) + out1_biases
    
    # Training computation.
    logits = forward_prop(tf_train_dataset)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset))
    test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset))


In [12]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary 
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 326.641144
Minibatch accuracy: 8.6%
Validation accuracy: 27.5%
Minibatch loss at step 500: 17.894100
Minibatch accuracy: 78.1%
Validation accuracy: 81.1%
Minibatch loss at step 1000: 7.557168
Minibatch accuracy: 86.7%
Validation accuracy: 80.1%
Minibatch loss at step 1500: 6.811553
Minibatch accuracy: 84.4%
Validation accuracy: 78.5%
Minibatch loss at step 2000: 5.913237
Minibatch accuracy: 79.7%
Validation accuracy: 80.0%
Minibatch loss at step 2500: 6.468968
Minibatch accuracy: 83.6%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 4.981013
Minibatch accuracy: 82.0%
Validation accuracy: 81.1%
Test accuracy: 88.3%
